In [1]:
# Generic inputs for most ML tasks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
import xgboost as xgb

pd.options.display.float_format = '{:,.2f}'.format

# setup interactive notebook mode
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML
from pandas.tseries.offsets import BDay


#### Read and pre-process data

In [2]:
# fetch data 

stock_data = pd.read_csv('Kaggle_Data/SP500_data.csv',parse_dates=['Date'],index_col='Date')

stock_data.head()

/tmp/ipykernel_27148/3165063347.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  stock_data = pd.read_csv('Kaggle_Data/SP500_data.csv',parse_dates=['Date'],index_col='Date')


,Close/Last,Open,High,Low,VIX_Open,VIX_High,VIX_Low,VIX_Close
Date,,,,,,,,
2022-04-01,"4,545.86","4,540.32","4,548.70","4,507.57",20.62,20.86,19.41,19.63
2022-04-04,"4,582.64","4,547.97","4,583.50","4,539.21",20.75,20.78,18.45,18.57
2022-04-05,"4,525.12","4,572.45","4,593.45","4,514.17",18.79,21.57,18.55,21.03
2022-04-06,"4,481.15","4,494.17","4,503.94","4,450.04",21.27,24.78,21.24,22.10
2022-04-07,"4,500.21","4,474.65","4,521.16","4,450.30",21.97,23.82,21.12,21.55


In [3]:
len(stock_data)

subset_data = stock_data
for i in range(1, 5):
    subset_data[f'Close_lag_{i}'] = subset_data['Close/Last'].shift(i)
    subset_data[f'Open_lag_{i}'] = subset_data['Open'].shift(i)
    subset_data[f'High_lag_{i}'] = subset_data['High'].shift(i)
    subset_data[f'Low_lag_{i}'] = subset_data['Low'].shift(i)
    subset_data[f'VIX_Open_lag_{i}'] = subset_data['Low'].shift(i)
    subset_data[f'VIX_High_lag_{i}'] = subset_data['Low'].shift(i)
    subset_data[f'VIX_Low_lag_{i}'] = subset_data['Low'].shift(i)
    subset_data[f'VIX_Close_lag_{i}'] = subset_data['Low'].shift(i)


subset_data = subset_data.dropna()
subset_data.head()

len(subset_data)

745

,Close/Last,Open,High,Low,VIX_Open,VIX_High,VIX_Low,VIX_Close,Close_lag_1,Open_lag_1,...,VIX_Low_lag_3,VIX_Close_lag_3,Close_lag_4,Open_lag_4,High_lag_4,Low_lag_4,VIX_Open_lag_4,VIX_High_lag_4,VIX_Low_lag_4,VIX_Close_lag_4
Date,,,,,,,,,,,,,,,,,,,,,
2022-04-07,"4,500.21","4,474.65","4,521.16","4,450.30",21.97,23.82,21.12,21.55,"4,481.15","4,494.17",...,"4,539.21","4,539.21","4,545.86","4,540.32","4,548.70","4,507.57","4,507.57","4,507.57","4,507.57","4,507.57"
2022-04-08,"4,488.28","4,494.15","4,520.41","4,474.60",21.25,22.34,20.28,21.16,"4,500.21","4,474.65",...,"4,514.17","4,514.17","4,582.64","4,547.97","4,583.50","4,539.21","4,539.21","4,539.21","4,539.21","4,539.21"
2022-04-11,"4,412.53","4,462.64","4,464.35","4,408.38",23.09,24.42,22.09,24.37,"4,488.28","4,494.15",...,"4,450.04","4,450.04","4,525.12","4,572.45","4,593.45","4,514.17","4,514.17","4,514.17","4,514.17","4,514.17"
2022-04-12,"4,397.45","4,437.59","4,471.00","4,381.34",24.94,25.38,22.27,24.26,"4,412.53","4,462.64",...,"4,450.30","4,450.30","4,481.15","4,494.17","4,503.94","4,450.04","4,450.04","4,450.04","4,450.04","4,450.04"
2022-04-13,"4,446.59","4,394.30","4,453.92","4,392.70",23.52,24.45,21.37,21.82,"4,397.45","4,437.59",...,"4,474.60","4,474.60","4,500.21","4,474.65","4,521.16","4,450.30","4,450.30","4,450.30","4,450.30","4,450.30"


741

In [4]:
subset_data.isna().sum()

Close/Last         0
Open               0
High               0
Low                0
VIX_Open           0
VIX_High           0
VIX_Low            0
VIX_Close          0
Close_lag_1        0
Open_lag_1         0
High_lag_1         0
Low_lag_1          0
VIX_Open_lag_1     0
VIX_High_lag_1     0
VIX_Low_lag_1      0
VIX_Close_lag_1    0
Close_lag_2        0
Open_lag_2         0
High_lag_2         0
Low_lag_2          0
VIX_Open_lag_2     0
VIX_High_lag_2     0
VIX_Low_lag_2      0
VIX_Close_lag_2    0
Close_lag_3        0
Open_lag_3         0
High_lag_3         0
Low_lag_3          0
VIX_Open_lag_3     0
VIX_High_lag_3     0
VIX_Low_lag_3      0
VIX_Close_lag_3    0
Close_lag_4        0
Open_lag_4         0
High_lag_4         0
Low_lag_4          0
VIX_Open_lag_4     0
VIX_High_lag_4     0
VIX_Low_lag_4      0
VIX_Close_lag_4    0
dtype: int64

In [5]:
subset_data.columns

Index(['Close/Last', 'Open', 'High', 'Low', 'VIX_Open', 'VIX_High', 'VIX_Low',
       'VIX_Close', 'Close_lag_1', 'Open_lag_1', 'High_lag_1', 'Low_lag_1',
       'VIX_Open_lag_1', 'VIX_High_lag_1', 'VIX_Low_lag_1', 'VIX_Close_lag_1',
       'Close_lag_2', 'Open_lag_2', 'High_lag_2', 'Low_lag_2',
       'VIX_Open_lag_2', 'VIX_High_lag_2', 'VIX_Low_lag_2', 'VIX_Close_lag_2',
       'Close_lag_3', 'Open_lag_3', 'High_lag_3', 'Low_lag_3',
       'VIX_Open_lag_3', 'VIX_High_lag_3', 'VIX_Low_lag_3', 'VIX_Close_lag_3',
       'Close_lag_4', 'Open_lag_4', 'High_lag_4', 'Low_lag_4',
       'VIX_Open_lag_4', 'VIX_High_lag_4', 'VIX_Low_lag_4', 'VIX_Close_lag_4'],
      dtype='object')

In [6]:
# Define features and target
X = subset_data[[f'Close_lag_{i}' for i in range(1, 5)] + 
                [f'Open_lag_{i}' for i in range(1, 5)] + 
                [f'High_lag_{i}' for i in range(1, 5)] +  
                [f'Low_lag_{i}' for i in range(1, 5)] + 
                [f'VIX_Open_lag_{i}' for i in range(1, 5)] +
                [f'VIX_High_lag_{i}' for i in range(1, 5)] + 
                [f'VIX_Low_lag_{i}' for i in range(1, 5)] + 
                [f'VIX_Close_lag_{i}' for i in range(1, 5)] + 
                ['Open']].copy()

y = subset_data['Close/Last']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

X_train
X_test
y_train
y_test

,Close_lag_1,Close_lag_2,Close_lag_3,Close_lag_4,Open_lag_1,Open_lag_2,Open_lag_3,Open_lag_4,High_lag_1,High_lag_2,...,VIX_High_lag_4,VIX_Low_lag_1,VIX_Low_lag_2,VIX_Low_lag_3,VIX_Low_lag_4,VIX_Close_lag_1,VIX_Close_lag_2,VIX_Close_lag_3,VIX_Close_lag_4,Open
Date,,,,,,,,,,,,,,,,,,,,,
2022-04-07,"4,481.15","4,525.12","4,582.64","4,545.86","4,494.17","4,572.45","4,547.97","4,540.32","4,503.94","4,593.45",...,"4,507.57","4,450.04","4,514.17","4,539.21","4,507.57","4,450.04","4,514.17","4,539.21","4,507.57","4,474.65"
2022-04-08,"4,500.21","4,481.15","4,525.12","4,582.64","4,474.65","4,494.17","4,572.45","4,547.97","4,521.16","4,503.94",...,"4,539.21","4,450.30","4,450.04","4,514.17","4,539.21","4,450.30","4,450.04","4,514.17","4,539.21","4,494.15"
2022-04-11,"4,488.28","4,500.21","4,481.15","4,525.12","4,494.15","4,474.65","4,494.17","4,572.45","4,520.41","4,521.16",...,"4,514.17","4,474.60","4,450.30","4,450.04","4,514.17","4,474.60","4,450.30","4,450.04","4,514.17","4,462.64"
2022-04-12,"4,412.53","4,488.28","4,500.21","4,481.15","4,462.64","4,494.15","4,474.65","4,494.17","4,464.35","4,520.41",...,"4,450.04","4,408.38","4,474.60","4,450.30","4,450.04","4,408.38","4,474.60","4,450.30","4,450.04","4,437.59"
2022-04-13,"4,397.45","4,412.53","4,488.28","4,500.21","4,437.59","4,462.64","4,494.15","4,474.65","4,471.00","4,464.35",...,"4,450.30","4,381.34","4,408.38","4,474.60","4,450.30","4,381.34","4,408.38","4,474.60","4,450.30","4,394.30"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-09,"5,319.31","5,199.50","5,240.03","5,186.33","5,252.57","5,293.13","5,206.42","5,151.14","5,328.03","5,330.64",...,"5,119.26","5,233.85","5,195.54","5,193.56","5,119.26","5,233.85","5,195.54","5,193.56","5,119.26","5,314.66"
2024-08-12,"5,344.16","5,319.31","5,199.50","5,240.03","5,314.66","5,252.57","5,293.13","5,206.42","5,358.67","5,328.03",...,"5,193.56","5,300.84","5,233.85","5,195.54","5,193.56","5,300.84","5,233.85","5,195.54","5,193.56","5,351.88"
2024-08-13,"5,344.39","5,344.16","5,319.31","5,199.50","5,351.88","5,314.66","5,252.57","5,293.13","5,371.20","5,358.67",...,"5,195.54","5,324.37","5,300.84","5,233.85","5,195.54","5,324.37","5,300.84","5,233.85","5,195.54","5,376.98"


,Close_lag_1,Close_lag_2,Close_lag_3,Close_lag_4,Open_lag_1,Open_lag_2,Open_lag_3,Open_lag_4,High_lag_1,High_lag_2,...,VIX_High_lag_4,VIX_Low_lag_1,VIX_Low_lag_2,VIX_Low_lag_3,VIX_Low_lag_4,VIX_Close_lag_1,VIX_Close_lag_2,VIX_Close_lag_3,VIX_Close_lag_4,Open
Date,,,,,,,,,,,,,,,,,,,,,
2024-08-16,"5,543.22","5,455.21","5,434.43","5,344.39","5,501.13","5,442.36","5,376.98","5,351.88","5,546.23","5,463.22",...,"5,324.37","5,501.13","5,415.91","5,376.98","5,324.37","5,501.13","5,415.91","5,376.98","5,324.37","5,530.50"
2024-08-19,"5,554.25","5,543.22","5,455.21","5,434.43","5,530.50","5,501.13","5,442.36","5,376.98","5,561.98","5,546.23",...,"5,376.98","5,525.17","5,501.13","5,415.91","5,376.98","5,525.17","5,501.13","5,415.91","5,376.98","5,557.23"
2024-08-20,"5,608.25","5,554.25","5,543.22","5,455.21","5,557.23","5,530.50","5,501.13","5,442.36","5,608.30","5,561.98",...,"5,415.91","5,550.74","5,525.17","5,501.13","5,415.91","5,550.74","5,525.17","5,501.13","5,415.91","5,602.88"
2024-08-21,"5,597.12","5,608.25","5,554.25","5,543.22","5,602.88","5,557.23","5,530.50","5,501.13","5,620.51","5,608.30",...,"5,501.13","5,585.50","5,550.74","5,525.17","5,501.13","5,585.50","5,550.74","5,525.17","5,501.13","5,603.09"
2024-08-22,"5,620.85","5,597.12","5,608.25","5,554.25","5,603.09","5,602.88","5,557.23","5,530.50","5,632.68","5,620.51",...,"5,525.17","5,591.57","5,585.50","5,550.74","5,525.17","5,591.57","5,585.50","5,550.74","5,525.17","5,637.77"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-17,"5,638.94","5,521.52","5,599.30","5,572.07","5,563.85","5,594.45","5,624.84","5,603.65","5,645.27","5,597.78",...,"5,528.41","5,563.85","5,504.65","5,546.09","5,528.41","5,563.85","5,504.65","5,546.09","5,528.41","5,635.60"
2025-03-18,"5,675.12","5,638.94","5,521.52","5,599.30","5,635.60","5,563.85","5,594.45","5,624.84","5,703.52","5,645.27",...,"5,546.09","5,631.12","5,563.85","5,504.65","5,546.09","5,631.12","5,563.85","5,504.65","5,546.09","5,654.53"
2025-03-19,"5,614.66","5,675.12","5,638.94","5,521.52","5,654.53","5,635.60","5,563.85","5,594.45","5,654.53","5,703.52",...,"5,504.65","5,597.76","5,631.12","5,563.85","5,504.65","5,597.76","5,631.12","5,563.85","5,504.65","5,632.37"


Date
2022-04-07   4,500.21
2022-04-08   4,488.28
2022-04-11   4,412.53
2022-04-12   4,397.45
2022-04-13   4,446.59
               ...   
2024-08-09   5,344.16
2024-08-12   5,344.39
2024-08-13   5,434.43
2024-08-14   5,455.21
2024-08-15   5,543.22
Name: Close/Last, Length: 592, dtype: float64

Date
2024-08-16   5,554.25
2024-08-19   5,608.25
2024-08-20   5,597.12
2024-08-21   5,620.85
2024-08-22   5,570.64
               ...   
2025-03-17   5,675.12
2025-03-18   5,614.66
2025-03-19   5,675.29
2025-03-20   5,662.89
2025-03-21   5,667.56
Name: Close/Last, Length: 149, dtype: float64

In [8]:
closelast_model = LinearRegression(fit_intercept = True)
#model = Ridge(alpha = 0.1, fit_intercept = True)
closelast_model.fit(X_train, y_train) 

# The following gives the R-square score
closelast_model.score(X_train, y_train) 

# This is the coefficient Beta_1, ..., Beta_7
closelast_model.coef_

# This is the coefficient Beta_0
closelast_model.intercept_

LinearRegression()

0.995047575365143

array([-0.36922603, -0.13981876, -0.09563426,  0.08316001,  0.04477974,
        0.07323973,  0.22527556,  0.25943373, -0.0433555 ,  0.1029635 ,
       -0.11877369, -0.25676782,  0.02789929, -0.00246595,  0.00364267,
       -0.04081866,  0.02789929, -0.00246595,  0.00364267, -0.04081866,
        0.02789929, -0.00246595,  0.00364267, -0.04081866,  0.02789929,
       -0.00246595,  0.00364267, -0.04081866,  0.02789929, -0.00246595,
        0.00364267, -0.04081866,  1.28736065])

34.10884503080615

In [9]:
training_residuals = y_train - closelast_model.predict(X_train)

In [10]:
rf = RandomForestRegressor(random_state=50, min_samples_leaf = 2, max_features = "sqrt")

rf = rf.fit(X_train, training_residuals) 
X_train.columns
rf.feature_importances_

Index(['Close_lag_1', 'Close_lag_2', 'Close_lag_3', 'Close_lag_4',
       'Open_lag_1', 'Open_lag_2', 'Open_lag_3', 'Open_lag_4', 'High_lag_1',
       'High_lag_2', 'High_lag_3', 'High_lag_4', 'Low_lag_1', 'Low_lag_2',
       'Low_lag_3', 'Low_lag_4', 'VIX_Open_lag_1', 'VIX_Open_lag_2',
       'VIX_Open_lag_3', 'VIX_Open_lag_4', 'VIX_High_lag_1', 'VIX_High_lag_2',
       'VIX_High_lag_3', 'VIX_High_lag_4', 'VIX_Low_lag_1', 'VIX_Low_lag_2',
       'VIX_Low_lag_3', 'VIX_Low_lag_4', 'VIX_Close_lag_1', 'VIX_Close_lag_2',
       'VIX_Close_lag_3', 'VIX_Close_lag_4', 'Open'],
      dtype='object')

array([0.04190091, 0.02995345, 0.02736719, 0.034314  , 0.02669957,
       0.02893484, 0.03006308, 0.04293753, 0.0318717 , 0.03414862,
       0.03005346, 0.03790188, 0.02795668, 0.02792039, 0.02852356,
       0.03182212, 0.02895523, 0.02653109, 0.02568   , 0.03327789,
       0.02628427, 0.02460509, 0.03068377, 0.02703004, 0.02728895,
       0.02915301, 0.0284054 , 0.02743007, 0.02631839, 0.02915523,
       0.02706757, 0.03313438, 0.03663066])

In [11]:
pred_residuals = rf.predict(X_test)
y_pred = pred_residuals + closelast_model.predict(X_test)

In [12]:
test_output = pd.DataFrame(y_pred, index = X_test.index, columns = ['pred_Close/Last'])
test_output = test_output.merge(y_test, left_index = True, right_index = True)
test_output.tail()
mean_absolute_error = abs(test_output['pred_Close/Last'] - test_output['Close/Last']).mean()
print('Mean absolute error is ')
print(mean_absolute_error)
abs(test_output['pred_Close/Last'] - test_output['Close/Last']).mean()/test_output['Close/Last'].mean()


,pred_Close/Last,Close/Last
Date,,
2025-03-17,"5,633.17","5,675.12"
2025-03-18,"5,639.85","5,614.66"
2025-03-19,"5,626.98","5,675.29"
2025-03-20,"5,609.65","5,662.89"
2025-03-21,"5,600.64","5,667.56"


Mean absolute error is 
35.97020123575871


0.006148516677778745

In [13]:
latest_data = pd.DataFrame([X.iloc[-1].values], columns=X.columns)
predictions = []
predicted_dates = [subset_data.index[-1] + BDay(i) for i in range(1, 5)]

for i in range(4):
    next_day_prediction = rf.predict(latest_data)[0] + closelast_model.predict(latest_data)[0]
    predictions.append(next_day_prediction)
    latest_data = np.roll(latest_data, -2)
    latest_data[0, -1] = next_day_prediction

# Print the predictions
print("Predicted Closing Prices for the Next 4 Business Days:")
for i, pred in enumerate(predictions):
    print(f"{predicted_dates[i].date()}: {pred}")

Predicted Closing Prices for the Next 4 Business Days:
2025-03-24: 5600.641043599412
2025-03-25: 5621.685594476108
2025-03-26: 5637.346627128926
2025-03-27: 5615.654577025059


/home/kevin-mesta/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/home/kevin-mesta/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/home/kevin-mesta/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/home/kevin-mesta/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/home/kevin-mesta/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/home